# Implémentation des bibliothèques

In [42]:
#!pip install import-ipynb


In [43]:
import pandas as pd  # Manipulation de données
import numpy as np  # Calculs numériques
import matplotlib.pyplot as plt  # Visualisation de données
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import seaborn as sns  # Visualisation avancée
import sklearn
import random
from datetime import datetime
import seaborn as sns
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import RandomOverSampler
# Importation des bibliothèques de scikit-learn pour la modélisation
from sklearn.model_selection import train_test_split  # Division des données en ensembles d'entraînement et de test
from sklearn.ensemble import RandomForestClassifier  # Modèle de classification Random Forest
from sklearn.linear_model import LinearRegression  # Modèle de régression linéaire
from sklearn.pipeline import make_pipeline  # Construction de pipelines pour le prétraitement et les modèles
from sklearn.preprocessing import StandardScaler, PolynomialFeatures  # Normalisation et transformation polynomiale
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer  # Imputation des valeurs manquantes
from sklearn.decomposition import PCA  # Réduction de dimension avec PCA
from sklearn.metrics import accuracy_score, classification_report   # Évaluation des modèles
from sklearn.metrics import confusion_matrix

# Import des outils de scikit-learn nécessaires
from sklearn.model_selection import GridSearchCV  # Optimisation des hyperparamètres
from sklearn.metrics import roc_auc_score, roc_curve  # Évaluation avec AUC-ROC
from sklearn.model_selection import cross_val_score  # Validation croisée
from sklearn.metrics import mean_squared_error

Un **Random Forest Classifier** repose su les principes suivants en partant de ** notre base de données** :
- **Bagging (Bootstrap Aggregation)** : Échantillonnage aléatoire avec remplacement.  
- **Arbres de décision (Decision Trees)** : Classificateurs individuels. (Random Subspace Method)
- **Sélection aléatoire des features** : Chaque arbre utilise un sous-ensemble des caractéristiques.  

# Importation des données

In [44]:
# Charger le fichier Excel
# df = pd.read_excel("C:\\Users\\anton\\Downloads\\sncf_data.xlsx")  
df = pd.read_excel("C:\\Users\\\scoup\\Downloads\\sncf_data.xlsx")
# Convertir en CSV
df.to_csv("data_csv", index=False, encoding="utf-8")
# On ne prend pas en compte les colonnes "qté cible" et "product policy" (trop de valeurs manquantes)*
df = df.drop(columns=['qté cible','product policy (short-term vision year - 2023)'])
df.fillna(0, inplace=True)  # Remplace les NaN par 0

# Convertir les colonnes de dates en nombre de jours depuis aujourd'hui
for col in df.select_dtypes(include=['datetime64']):
    df[col] = (datetime.today() - df[col]).dt.days 
    
# Séparer les features (X) et la cible (y)
target_column = "label"  # Remplace par le nom de ta colonne cible
X = df.drop(columns=[target_column])
y = df[target_column]

X

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\scoup\AppData\Local\Temp\ipykernel_26276\933080086.py:3: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_excel("C:\\Users\\\scoup\\Downloads\\sncf_data.xlsx")


,symbol,supplier,serial letter,repair complexity (0NA),evaluation ratio between repair cost and cost of new (0NA),\nexisting substitute product (0NA),component having a role in security (0NA),does the product/block if the component have a role in security? (0NA),several components involved? (0NA),is the product recent? (<10 years) (0NA),\nis the cost of inventory expensive (0NA),substitution component not available (0NA),do we have equipment to carry out approval tests? (0NA),have we done a technical validation? (0NA),generally at least equivalent criterion,active quantity,quantity of existing stock at SNCF?,"no longer supply (new or used), no longer supply new, reparable, supply new",processing date
0,79544181,SCLE,AB,1,1,1,1,1,1,0,0.0,0,0,0,1,1050,837,supply new,602
1,79402561,SCLE,BA,0,0,0,0,0,1,0,0.0,1,1,1,1,436,999,supply new,596
2,79540265,SCLE,BA,0,0,1,0,1,0,0,0.0,0,1,1,1,70,1655,supply new,940
3,79540265,SCLE,BB,0,0,1,0,1,0,0,0.0,0,1,1,1,149,1655,supply new,940
4,79540265,SCLE,BBM,0,0,1,0,1,0,0,0.0,0,1,1,1,24,1655,supply new,940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,79545633,HITACHI,B__,1,0,0,1,1,1,0,0.0,1,1,1,0,0,876,supply new,2189
313,79545464,Alstom,AAA,0,0,0,1,0,0,0,0.0,0,1,1,1,6844,2305,supply new,5535
314,79545106,Alstom,AE,0,0,0,1,1,1,0,0.0,0,1,1,1,30,3583,supply new,891
315,79545106,Alstom,AE,0,0,0,1,1,1,0,0.0,0,1,1,1,30,3583,supply new,891


In [45]:
# Stocker la colonne "symbol" pour l'afficher plus tard
X["symbol"] = df["symbol"]
categorical_features = X.nunique()[X.nunique() == 2].index.tolist()
# Encodage des variables catégorielles si nécessaire
X = pd.get_dummies(X)
# je crois il faut pas supprimer la première colonne
# X = pd.get_dummies(X)

X["\nis the cost of inventory expensive (0NA)"]=X["\nis the cost of inventory expensive (0NA)"].astype(int)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Columns: 114 entries, symbol to no longer supply (new or used), no longer supply new, reparable, supply new_supply new
dtypes: bool(98), int64(16)
memory usage: 70.1 KB


# Fonction test_train_split

In [46]:
def custom_train_test_split_df(X, y, test_size, random_state, stratify=None):
    np.random.seed(random_state)
    # Vérifie si `X` est un DataFrame pour conserver les noms des colonnes.
    if isinstance(X, pd.DataFrame):
        columns = X.columns  # Sauvegarder les noms des colonnes
    else:
        columns = None  # Gérer les cas où X n'est pas un DataFrame
    
    X = np.array(X)
    y = np.array(y)
    
    if stratify is not None:
        # Séparation stratifiée
        unique_classes, class_counts = np.unique(y, return_counts=True)
        X_train, X_test, y_train, y_test = [], [], [], []
        
        for class_label, count in zip(unique_classes, class_counts):
            # Sélectionner les indices des échantillons de la classe actuelle
            indices = np.where(y == class_label)[0]
            np.random.shuffle(indices)
            test_count = int(count * test_size)
            test_indices = indices[:test_count]
            train_indices = indices[test_count:]
            # Ajouter les échantillons aux ensembles d'entraînement et de test
            X_train.extend(X[train_indices])
            X_test.extend(X[test_indices])
            y_train.extend(y[train_indices])
            y_test.extend(y[test_indices])
        
        X_train, X_test = np.array(X_train), np.array(X_test)
        y_train, y_test = np.array(y_train), np.array(y_test)
    
    else:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        test_count = int(len(X) * test_size)
        test_indices = indices[:test_count]
        train_indices = indices[test_count:]
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
    
    if columns is not None:
        df_train = pd.DataFrame(X_train, columns=columns)
        df_test = pd.DataFrame(X_test, columns=columns)
    else:
        df_train = pd.DataFrame(X_train)
        df_test = pd.DataFrame(X_test)
    # Ajouter la colonne cible au train et test
    df_train['label'] = y_train
    df_test['label'] = y_test
    
    return df_train, df_test

In [47]:
df_train,df_test = custom_train_test_split_df(X, y, test_size=0.2,random_state=42,stratify=y)
len(df_train)
df_train.info()
df_train.head()

df_train.columns = df_train.columns.str.replace(' ', '').str.replace('\n', '')
df_test.columns = df_test.columns.str.replace(' ', '').str.replace('\n', '')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Columns: 115 entries, symbol to label
dtypes: object(115)
memory usage: 230.1+ KB


In [48]:
df_train.columns 

Index(['symbol', 'repaircomplexity(0NA)',
       'evaluationratiobetweenrepaircostandcostofnew(0NA)',
       'existingsubstituteproduct(0NA)', 'componenthavingaroleinsecurity(0NA)',
       'doestheproduct/blockifthecomponenthavearoleinsecurity?(0NA)',
       'severalcomponentsinvolved?(0NA)', 'istheproductrecent?(<10years)(0NA)',
       'isthecostofinventoryexpensive(0NA)',
       'substitutioncomponentnotavailable(0NA)',
       ...
       'serialletter_EBH', 'serialletter_EC', 'serialletter_ECF',
       'serialletter_FA', 'serialletter_FB', 'serialletter____',
       'nolongersupply(neworused),nolongersupplynew,reparable,supplynew_nolongersupplynew',
       'nolongersupply(neworused),nolongersupplynew,reparable,supplynew_reparable',
       'nolongersupply(neworused),nolongersupplynew,reparable,supplynew_supplynew',
       'label'],
      dtype='object', length=115)

# Boostrapping

In [49]:
def bootstrapping(train_df, n_bootstrap):
    bootstrap_indices = np.random.randint(low=0, high=len(train_df), size=n_bootstrap)
    df_bootstrapped = train_df.iloc[bootstrap_indices]
    return df_bootstrapped

In [50]:
df_boostrapping = bootstrapping(df_train, 10)
df_boostrapping

,symbol,repaircomplexity(0NA),evaluationratiobetweenrepaircostandcostofnew(0NA),existingsubstituteproduct(0NA),componenthavingaroleinsecurity(0NA),doestheproduct/blockifthecomponenthavearoleinsecurity?(0NA),severalcomponentsinvolved?(0NA),istheproductrecent?(<10years)(0NA),isthecostofinventoryexpensive(0NA),substitutioncomponentnotavailable(0NA),...,serialletter_EBH,serialletter_EC,serialletter_ECF,serialletter_FA,serialletter_FB,serialletter____,"nolongersupply(neworused),nolongersupplynew,reparable,supplynew_nolongersupplynew","nolongersupply(neworused),nolongersupplynew,reparable,supplynew_reparable","nolongersupply(neworused),nolongersupplynew,reparable,supplynew_supplynew",label
20,79545017,0,0,0,1,1,0,1,0,1,...,False,False,False,False,False,False,False,False,True,LBO
47,79544362,0,0,0,1,1,0,1,0,1,...,False,False,False,False,False,False,False,False,True,redesign majeur
147,79520902,0,1,1,1,1,0,1,0,0,...,False,False,False,False,False,True,False,False,True,stock
247,79520904,0,0,0,0,1,1,0,0,0,...,False,False,False,False,False,False,False,True,False,substitution
127,79520541,0,0,0,1,1,1,0,0,1,...,False,False,False,False,False,False,False,True,False,redesign mineur
135,79520541,0,0,0,1,1,1,0,0,1,...,False,False,False,False,False,False,False,True,False,redesign mineur
134,79520541,0,0,0,1,1,1,0,0,1,...,False,False,False,False,False,False,False,True,False,redesign mineur
194,79565032,0,0,0,0,1,0,1,0,0,...,False,False,False,False,False,False,False,True,False,substitution
144,79520541,0,0,0,1,1,1,0,0,1,...,False,False,False,False,False,False,False,True,False,redesign mineur
127,79520541,0,0,0,1,1,1,0,0,1,...,False,False,False,False,False,False,False,True,False,redesign mineur


# Fonction permettant de distinguer les variables binaires(1/0 ou Vrai/Faux) et les variables continue (active quantity,quantity existing,processing date)

In [51]:
def determine_type_of_feature(df):
    
    feature_types = []
    # On considère qu'une variable catégorielle a moins de 2 valeurs uniques vu qu'on a fait un encodage one-hot
    n_unique_values_treshold = 2
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [52]:
determine_type_of_feature(df_train)

['continuous',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'continuous',
 'continuous',
 'continuous',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorical',
 'categorica

# Fonction d'arbre de decision 

## Purete d'une feuille

In [53]:
# Une feuille est-elle pure 
def check_purity(data):
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)
    # il y a qu'une seule classe
    if len(unique_classes) == 1:
        return True
    else:
        return False

In [54]:
# Création d'exemples de jeux de données pour tester la fonction check_purity()

# Jeu de données où toutes les étiquettes sont identiques (pures)
data_pure = np.array([
    [5.1, 3.5, 1.4, 0.2, "Setosa"],
    [4.9, 3.0, 1.4, 0.2, "Setosa"],
    [4.7, 3.2, 1.3, 0.2, "Setosa"],
    [4.6, 3.1, 1.5, 0.2, "Setosa"]
])

# Jeu de données contenant différentes classes (mélangées)
data_mixed = np.array([
    [5.1, 3.5, 1.4, 0.2, "Setosa"],
    [7.0, 3.2, 4.7, 1.4, "Versicolor"],
    [6.3, 3.3, 6.0, 2.5, "Virginica"],
    [4.9, 3.0, 1.4, 0.2, "Setosa"]
])


# Test de la fonction
pure_test = check_purity(data_pure)  # Doit retourner True
mixed_test = check_purity(data_mixed)  # Doit retourner False

pure_test, mixed_test

(True, False)

## Determine la classe majoritaire

In [55]:
def class_majoritaire(data):
    label_column = data[:, -1]
    unique_classes, count_unique_classes = np.unique(label_column, return_counts=True)
    # on prend la classe majoritaire
    index = count_unique_classes.argmax()
    class_majoritaire = unique_classes[index]
    return class_majoritaire

In [56]:
# Création d'exemples de jeux de données pour tester la fonction class_majoritaire()

# Jeu de données avec une classe majoritaire évidente
data_majority_A = np.array([
    [5.1, 3.5, 1.4, 0.2, "A"],
    [4.9, 3.0, 1.4, 0.2, "A"],
    [4.7, 3.2, 1.3, 0.2, "B"],
    [4.6, 3.1, 1.5, 0.2, "A"],
    [5.0, 3.4, 1.6, 0.4, "A"],
    [5.2, 3.5, 1.5, 0.2, "B"]
])

# Jeu de données avec une autre classe majoritaire
data_majority_B = np.array([
    [6.1, 2.8, 4.7, 1.2, "B"],
    [5.7, 2.5, 5.0, 2.0, "B"],
    [6.0, 2.9, 4.5, 1.5, "A"],
    [6.3, 3.3, 6.0, 2.5, "B"],
    [6.4, 2.8, 5.6, 2.1, "B"],
    [6.2, 2.8, 4.8, 1.8, "A"]
])


# Test de la fonction
majority_A = class_majoritaire(data_majority_A)  # Doit retourner "A"
majority_B = class_majoritaire(data_majority_B)  # Doit retourner "B"

majority_A, majority_B

(np.str_('A'), np.str_('B'))

In [57]:
# Quelles sont les séparartions possible dans chaque colonne? 
def get_potential_splits(data):
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):  # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

In [58]:
# Création d'un exemple de jeu de données pour tester get_potential_splits()

data_example = np.array([
    [5.1, 3.5, 1.4, "A"],
    [4.9, 3.0, 1.4, "B"],
    [4.7, 3.2, 1.3, "A"],
    [4.6, 3.1, 1.5, "B"],
    [5.0, 3.4, 1.6, "A"]
])


# Test de la fonction
potential_splits_example = get_potential_splits(data_example)
potential_splits_example


{0: array(['4.6', '4.7', '4.9', '5.0', '5.1'], dtype='<U32'),
 1: array(['3.0', '3.1', '3.2', '3.4', '3.5'], dtype='<U32'),
 2: array(['1.3', '1.4', '1.5', '1.6'], dtype='<U32')}

In [59]:
# 1.4 Calcul de l'impurete de gini
def calculate_gini(data):
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)
    # probabilité de chaque classe
    probabilities = counts / counts.sum()
    # gini
    gini = 1 - sum(probabilities ** 2)
    
    return gini

In [60]:
# Exemple
data = np.array([
    [1, 'A'],
    [2, 'A'],
    [3, 'B'],
    [4, 'B'],
    [5, 'B']
])

gini_impurity = calculate_gini(data)
print("Gini Impurity:", gini_impurity)

Gini Impurity: 0.48


In [61]:
# calcul de l'impurete gini pour un split
def calculate_overall_gini(data_below, data_above):
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n
    
    overall_gini = (p_data_below * calculate_gini(data_below) 
                    + p_data_above * calculate_gini(data_above))
    
    return overall_gini

In [62]:
# Example data
data = np.array([
    [1, 'A'],
    [2, 'A'],
    [3, 'B'],
    [4, 'B'],
    [5, 'B']
],dtype=object)

# Split the data
data[:, 0]=data[:,0].astype(int)
data_below = data[data[:, 0] <= 3]
data_above = data[data[:, 0] > 3]

# Calculate overall Gini impurity
overall_gini = calculate_overall_gini(data_below, data_above)
print("Overall Gini Impurity:", overall_gini)

Overall Gini Impurity: 0.26666666666666666


### Faire les separations pour diminuer l'impurete de gini

In [63]:
# Split data
def split_data(data, split_column, split_value):
    split_column_values = data[:, split_column]
    
    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # Feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [64]:
# Création d'un exemple de jeu de données pour tester split_data()

# Définition des types de features
FEATURE_TYPES = ["continuous", "continuous", "categorical"]  # Les deux premières colonnes sont continues, la troisième est catégorielle

# Données factices avec des variables continues et catégorielles
data_example = np.array([
    [5.1, 3.5, "A"],
    [4.9, 3.0, "B"],
    [4.7, 3.2, "A"],
    [4.6, 3.1, "B"],
    [5.0, 3.4, "A"]
], dtype=object)  # Utilisation de dtype=object pour mélanger nombres et chaînes

# Test de la fonction avec une variable continue (colonne 0, valeur de split = 4.8)
data_below_cont, data_above_cont = split_data(data_example, split_column=0, split_value=4.8)

# Test de la fonction avec une variable catégorielle (colonne 2, valeur de split = "A")
data_below_cat, data_above_cat = split_data(data_example, split_column=2, split_value="A")

data_below_cont, data_above_cont



(array([[4.7, 3.2, 'A'],
        [4.6, 3.1, 'B']], dtype=object),
 array([[5.1, 3.5, 'A'],
        [4.9, 3.0, 'B'],
        [5.0, 3.4, 'A']], dtype=object))

In [65]:
data_below_cat, data_above_cat

(array([[5.1, 3.5, 'A'],
        [4.7, 3.2, 'A'],
        [5.0, 3.4, 'A']], dtype=object),
 array([[4.9, 3.0, 'B'],
        [4.6, 3.1, 'B']], dtype=object))

In [66]:
# Détermination du meilleur split en utilisant l'impureté de Gini
def determine_best_split(data, potential_splits):
    overall_gini = 9999  # Initialiser avec une valeur élevée
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_gini = calculate_overall_gini(data_below, data_above)
            
            if current_overall_gini <= overall_gini:
                overall_gini = current_overall_gini
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value


In [67]:
# Test de la fonction determine_best_split()
potential_splits = get_potential_splits(data_example)
best_column, best_value = determine_best_split(data_example, potential_splits)

best_column, best_value

(1, 3.1)

# Algorithme de construction de l'arbre de décision

In [68]:
def decision_tree_algorithm(df, counter=0, min_samples=1, max_depth=5, random_subspace=None):
    
    # Préparation des données
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    # Cas d'arrêt 
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = class_majoritaire(data)
        return classification

    # Sélection des features aléatoires pour Random Forest
    if random_subspace:
        feature_indices = np.random.choice(len(COLUMN_HEADERS) - 1, size=random_subspace, replace=False)
    else:
        feature_indices = range(len(COLUMN_HEADERS) - 1)  # Utiliser toutes les features sauf la cible

    # Incrémentation de la profondeur
    counter += 1

    # Sélection des meilleurs splits en fonction des features sélectionnées
    potential_splits = get_potential_splits(data)
    potential_splits = {k: v for k, v in potential_splits.items() if k in feature_indices}
    
    split_column, split_value = determine_best_split(data, potential_splits)
    data_below, data_above = split_data(data, split_column, split_value)
    
    # Vérifier si l'un des groupes est vide
    if len(data_below) == 0 or len(data_above) == 0:
        classification = class_majoritaire(data)
        return classification
    
    # Construire la question
    feature_name = COLUMN_HEADERS[split_column]
    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        question = "{} <= {}".format(feature_name, split_value)
    else:
        question = "{} = {}".format(feature_name, split_value)
    
    # Initialiser le sous-arbre
    sub_tree = {question: []}
    
    # Récursion pour construire les sous-arbres
    yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth, random_subspace)
    no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth, random_subspace)
    
    # Optimisation : Si les deux réponses sont identiques, on ne garde pas la question
    if yes_answer == no_answer:
        sub_tree = yes_answer
    else:
        sub_tree[question].append(yes_answer)
        sub_tree[question].append(no_answer)
    
    return sub_tree


In [69]:
# Exécution de l'algorithme avec le dataset factice
tree = decision_tree_algorithm(df_boostrapping, min_samples=1, max_depth=150)
question = list(tree.keys())[0]
tree[question]

['redesign mineur',
 {'nolongersupply(neworused),nolongersupplynew,reparable,supplynew_supplynew = True': [{'serialletter____ = True': ['stock',
     {'serialletter_AE = True': ['redesign majeur', 'LBO']}]},
   'substitution']}]

## Faire une prédiction

In [70]:
def parse_question(question):
    """Sépare une condition sous forme de texte en feature, opérateur et valeur."""
    parts = question.split(" ", 2)
    if len(parts) == 3:
        feature_name, comparison_operator, value = parts
    else:
        raise ValueError(f"Format incorrect pour la question: {question}")
    return feature_name.strip(), comparison_operator.strip(), value.strip()
feature_name, comparison_operator, value = parse_question(list(tree.keys())[0])
type(value)


str

Vérifier si la clé est un label final
    if question in ["substitution", "redesign mineur", "stock", "redesign majeur", "LBO"]:
        return question

In [71]:
def predict_example(example, tree):
    # extrait le noeud de l'arbre
    question = list(tree.keys())[0]
    # Divise les élèments de la racine, le nom, l'opérateur de comparaison et la valeur
    feature_name, comparison_operator, value = parse_question(question)
    # ask question
    if comparison_operator == "<=":
        if int(example[feature_name]) <= int(value):
            # on va a gauche de l'arbre
            answer = tree[question][0]
        else:
            # on va a droite de l'arbre
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            # on va a gauche de l'arbre
            answer = tree[question][0]
        else:
            # on va a droite de l'arbre
            answer = tree[question][1]

    # on a atteint une feuille car ce n'est pas un dictionnaire, on retourne la réponse
    if not isinstance(answer, dict):
        return answer
    
    # Sinon on continue de parcourir l'arbre sur le sous-arbre
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)


In [72]:
example = df_test.iloc[50]
example
X_test_example = example.drop("label")
X_test_example.info()

<class 'pandas.core.series.Series'>
Index: 114 entries, symbol to nolongersupply(neworused),nolongersupplynew,reparable,supplynew_supplynew
Series name: 50
Non-Null Count  Dtype 
--------------  ----- 
114 non-null    object
dtypes: object(1)
memory usage: 1.8+ KB


In [73]:
prediction = predict_example(X_test_example, tree)
prediction

'substitution'

# 3.2 Retourne la prédiction pour chaque ligne du set de test


In [74]:
def decision_tree_predictions(test_df, tree):
    predictions = test_df.apply(predict_example, args=(tree,), axis=1)
    return predictions

In [75]:
predictions = decision_tree_predictions(df_test, tree)
predictions

0                 LBO
1                 LBO
2                 LBO
3                 LBO
4                 LBO
           ...       
56                LBO
57    redesign mineur
58                LBO
59       substitution
60       substitution
Length: 61, dtype: object

# Application Arbre de décision

In [76]:
# Création de l'arbre de décision
def random_forest_algorithm(train_df, n_trees, n_bootstrap, n_features, dt_max_depth):
    forest = []
    for i in range(n_trees):
        df_bootstrapped = bootstrapping(train_df, n_bootstrap)
        tree = decision_tree_algorithm(df_bootstrapped, max_depth=dt_max_depth, random_subspace=n_features)
        forest.append(tree)
    
    return forest



In [77]:
forest = random_forest_algorithm(df_train, n_trees=4, n_bootstrap=800, n_features=None, dt_max_depth=150)
len(forest)

4

In [78]:
# Prédiction avec un random forest
def random_forest_predictions(df_test, forest):
    df_predictions = {}
    for i in range(len(forest)):
        column_name = "tree_{}".format(i)
        predictions = decision_tree_predictions(df_test, tree=forest[i])
        df_predictions[column_name] = predictions

    df_predictions = pd.DataFrame(df_predictions)
    random_forest_predictions = df_predictions.mode(axis=1).iloc[:, 0]
    return random_forest_predictions

In [79]:
predictions = random_forest_predictions(df_test, forest)
predictions

0              LBO
1              LBO
2              LBO
3              LBO
4              LBO
          ...     
56    substitution
57    substitution
58    substitution
59    substitution
60    substitution
Name: 0, Length: 61, dtype: object

# Accuracy

In [80]:
def accuracy_metric(y_test, y_predicted):
	correct = 0
	for i in range(len(y_test)):
		if y_test[i] == y_predicted[i]:
			correct += 1
	return correct / float(len(y_test)) * 100.0

In [81]:
accuracy = accuracy_metric(predictions, df_test.label)
print("Accuracy = {}".format(accuracy))

Accuracy = 96.72131147540983
